## **Bibliotecas**

In [0]:
import requests
import json
from datetime import datetime

## **Funções**

In [0]:
%run ./functions

## **Parâmetros**

In [0]:
bronze_path = lake_path

## **Extração**

In [0]:
#Exemplo de paginação: https://api.openbrewerydb.org/breweries?page=2&per_page=10

# URL da API
breweries_url = "https://api.openbrewerydb.org/breweries"
all_data = []

# Parâmetros de paginação
page = 1
per_page = 4000  

while True:
    # Requisição com os parâmetros de paginação
    response = requests.get(breweries_url, params={"page": page, "per_page": per_page})
    
    # Verifica a request
    if response.status_code != 200:
        print(f"Erro na requisição: {response.status_code}")
        break
    
    # Converter para JSON
    data = response.json()
    
    # Parar quando acabar os dados
    if not data:
        print(f"Total de páginas: {page - 1}")
        break
    
    # Add os dados da página na lista
    all_data.extend(data)
    
    # Adiciona a pr´xima página
    page += 1

Total de páginas: 42


In [0]:
all_data[0:4]

Out[31]: [{'id': '5128df48-79fc-4f0f-8b52-d06be54d0cec',
  'name': '(405) Brewing Co',
  'brewery_type': 'micro',
  'address_1': '1716 Topeka St',
  'address_2': None,
  'address_3': None,
  'city': 'Norman',
  'state_province': 'Oklahoma',
  'postal_code': '73069-8224',
  'country': 'United States',
  'longitude': '-97.46818222',
  'latitude': '35.25738891',
  'phone': '4058160490',
  'website_url': 'http://www.405brewing.com',
  'state': 'Oklahoma',
  'street': '1716 Topeka St'},
 {'id': '9c5a66c8-cc13-416f-a5d9-0a769c87d318',
  'name': '(512) Brewing Co',
  'brewery_type': 'micro',
  'address_1': '407 Radam Ln Ste F200',
  'address_2': None,
  'address_3': None,
  'city': 'Austin',
  'state_province': 'Texas',
  'postal_code': '78745-1197',
  'country': 'United States',
  'longitude': None,
  'latitude': None,
  'phone': '5129211545',
  'website_url': 'http://www.512brewing.com',
  'state': 'Texas',
  'street': '407 Radam Ln Ste F200'},
 {'id': '34e8c68b-6146-453f-a4b9-1f6cd99a5ada'

In [0]:
# Criação do Dataframe e da Columa Timestamp
df_breweries = (spark.createDataFrame(all_data).withColumn('tmstp_ingestion', lit(datetime.now())))

In [0]:
display(df_breweries.limit(5))

address_1,address_2,address_3,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,state_province,street,website_url,tmstp_ingestion
1716 Topeka St,null,null,micro,Norman,United States,5128df48-79fc-4f0f-8b52-d06be54d0cec,35.25738891,-97.46818222,(405) Brewing Co,4058160490,73069-8224,Oklahoma,Oklahoma,1716 Topeka St,http://www.405brewing.com,2024-09-26T20:39:35.532+0000
407 Radam Ln Ste F200,null,null,micro,Austin,United States,9c5a66c8-cc13-416f-a5d9-0a769c87d318,null,null,(512) Brewing Co,5129211545,78745-1197,Texas,Texas,407 Radam Ln Ste F200,http://www.512brewing.com,2024-09-26T20:39:35.532+0000
8100 Washington Ave,null,null,micro,Mount Pleasant,United States,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,42.72010826899558,-87.88336350209435,1 of Us Brewing Company,2624847553,53406-3920,Wisconsin,Wisconsin,8100 Washington Ave,https://www.1ofusbrewing.com,2024-09-26T20:39:35.532+0000
1501 E St,null,null,large,San Diego,United States,ef970757-fe42-416f-931d-722451f1f59c,32.714813,-117.129593,10 Barrel Brewing Co,6195782311,92101-6618,California,California,1501 E St,http://10barrel.com,2024-09-26T20:39:35.532+0000
62970 18th St,null,null,large,Bend,United States,6d14b220-8926-4521-8d19-b98a2d6ec3db,44.08683531,-121.281706,10 Barrel Brewing Co,5415851007,97701-9847,Oregon,Oregon,62970 18th St,http://www.10barrel.com,2024-09-26T20:39:35.532+0000


In [0]:
# Salvar na Bronze
(df_breweries.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f'{bronze_path}/bronze/tb_bronze_breweries'))